In [14]:
import numpy as np
import cv2
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
import numpy as np
from skimage import io
import random
from PIL import Image

In [2]:
def read_image(address):
    img = cv2.imread(address)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [32]:

img_1941 = read_image('inputs/P4/kobytev_1941.png')
img_1941 = img_1941[0:956, 0:771]

img_1945 = read_image('inputs/P4/kobytev_1945.png')
duration = 2
frame_rate = 30
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-r', str(frame_rate),'-s',str(img_1945.shape[1])+'x'+str(img_1945.shape[0]), '-i', '-', '-c:v', 'libx264', '-crf', '25','-vf','scale=trunc(iw/2)*2:trunc(ih/2)*2','-pix_fmt','yuv420p', "output1.mp4"], stdin=PIPE)

counts= int(duration*frame_rate)
for count in range(counts):
    alpha = count/(counts-1)
    beta = (1.0 - alpha)
    dst = np.uint8(alpha * (img_1945) + beta * (img_1941))
    output_address = "outputs/q4/p1/"+ str(count)+"_alpha_"+str(alpha)+".png"
    cv2.imwrite(output_address, dst)
    res = Image.fromarray(cv2.cvtColor(np.uint8(dst), cv2.COLOR_BGR2RGB))
    res.save(p.stdin,'JPEG')

p.stdin.close()
p.wait()



0

Part B:

In [4]:
def define_points_of_images(image,number_of_points=26):
    plt.imshow(image)
    points= plt.ginput(number_of_points,timeout=-1)
    plt.close()
    return points

In [27]:
def draw_circle(img,points):
    image = img
    for point in points:
        center_coordinates = (int(point[0]), int(point[1]))
        radius = 2
        color = (0, 0, 255)
        thickness = 2
        image = cv2.circle(image, center_coordinates, radius, color, thickness)
    output_address = "outputs/q4/.png"
    cv2.imwrite(output_address, image)

In [ ]:
def add_fix_points(list_points,shape):
    list_points.append((1,1))
    list_points.append((shape[1]-1,1))
    list_points.append(((shape[1]-1)//2,1))
    list_points.append((1,shape[0]-1))
    list_points.append((1,(shape[0]-1)//2))
    list_points.append(((shape[1]-1)//2,shape[0]-1))
    list_points.append((shape[1]-1,shape[0]-1))
    list_points.append(((shape[1]-1),(shape[0]-1)//2))

In [ ]:
def add_fix_points2(list_detail,shape):
    arr = list_detail/2
    arr = np.append(arr,[[1,1]],axis=0)
    arr = np.append(arr,[[shape[1]-1,1]],axis=0)
    arr = np.append(arr,[[(shape[1]-1)//2,1]],axis=0)
    arr = np.append(arr,[[1,shape[0]-1]],axis=0)
    arr = np.append(arr,[[1,(shape[0]-1)//2]],axis=0)
    arr = np.append(arr,[[(shape[1]-1)//2,shape[0]-1]],axis=0)
    arr = np.append(arr,[[shape[1]-1,shape[0]-1]],axis=0)
    arr = np.append(arr,[[(shape[1]-1),(shape[0]-1)//2]],axis=0)
    return arr

In [9]:
def define_points(imgList):
    list_detail = np.zeros((26,2))
    list_points_image_1 = []
    list_points_image_2 = []
    for w in range(len(imgList)):
        img = imgList[j]
        points = define_points_of_images(img)
        draw_circle(img,points)
        if(w == 0):
            list_points = list_points_image_1
        else:
            list_points = list_points_image_2
        for i in range(26):
            list_points.append((int(points[i][0]), int(points[i][1])))
            list_detail[i][0] += int(points[i][0])
            list_detail[i][1] += int(points[i][1])
        list_points = add_fix_points(list_points,(img.shape[0],img.shape[1]))
    combine = add_fix_points2(list_detail,(img.shape[0],img.shape[1]))
    
    return list_points_image_1,list_points_image_2,combine

In [ ]:
def is_contain(rectange, point):
    if point[0] < rectange[0] or point[1] < rectange[1] or point[0] > rectange[2] or point[1] > rectange[3]:
        return False
    return True

In [ ]:
def draw_delaunay(size, s, dic):
    list_of_d = [] 
    r = (0, 0,  size[0],  size[1])
    for t in s.getTriangleList() :
        point1 = (int(t[0]), int(t[1]))
        point2 = (int(t[2]), int(t[3]))
        point3 = (int(t[4]), int(t[5]))
        if is_contain(r, point1) and is_contain(r, point2) and is_contain(r, point3) :
            list_of_d.append((dic[point1],dic[point2],dic[point3]))
    dic = {}
    return list_of_d

In [10]:
def create_delaunay(size, lists):
    d = cv2.Subdiv2D((0, 0, size[1], size[0]))
    points = [(int(x[0]),int(x[1])) for x in lists]
    dictionary = {x[0]:x[1] for x in list(zip(points, range(26)))}
    for p in points:
        d.insert(p)
    list4 = draw_delaunay(size, d, dictionary)
    return list4

In [ ]:
def transform_rectangles(src, first, second, shape) :
    output = cv2.warpAffine(src, cv2.getAffineTransform(np.float32(first), np.float32(second)) , (shape[0], shape[1]) , None , flags=cv2.INTER_LINEAR , borderMode=cv2.BORDER_REFLECT_101)
    return output

In [ ]:
def morphology(img1, img2, img, t1, t2, t, alpha) :
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))
    t1Rectangels = []
    t2Rectangels = []
    tRectangels = []
    for i in range(3):
        tRectangels.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rectangels.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rectangels.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
    m = np.zeros((r[3], r[2], 3)).astype(float)
    cv2.fillConvexPoly(m, np.int32(tRectangels), (1.0, 1.0, 1.0), 16, 0)
    size = (r[2], r[3])
    wimage1 = transform_rectangles(img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]], t1Rectangels, tRectangels, size)
    wimage2 = transform_rectangles(img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]], t2Rectangels, tRectangels, size)
    final_img = (1.0 - alpha) * wimage1 + alpha * wimage2
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = m * final_img  + ( 1 - m ) * img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] 



In [18]:
def create_video(seconds,frame_rate,img1,img2,points1,points2,tri_list,size,output):
    counts = int(seconds*frame_rate)
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-r', str(frame_rate),'-s',str(size[1])+'x'+str(size[0]), '-i', '-', '-c:v', 'libx264', '-crf', '25','-vf','scale=trunc(iw/2)*2:trunc(ih/2)*2','-pix_fmt','yuv420p', output], stdin=PIPE)
    for j in range(0, counts):
        img1 = np.float32(img1)
        img2 = np.float32(img2)
        points = []
        alpha = j/(counts-1)
        for i in range(0, len(points1)):
            points.append(( alpha * points2[i][0] + (1 - alpha) * points1[i][0], alpha * points2[i][1] + (1 - alpha) * points1[i][1]))
        output = np.zeros(img1.shape).astype(img1.dtype)
        for i in range(len(tri_list)):    
            x = int(tri_list[i][0])
            y = int(tri_list[i][1])
            z = int(tri_list[i][2])  
            t1 = [points1[x], points1[y], points1[z]]
            t2 = [points2[x], points2[y], points2[z]]
            t = [points[x], points[y], points[z]]
            morphology(img1, img2, output, t1, t2, t, alpha)

        out_image = Image.fromarray(cv2.cvtColor(np.uint8(output), cv2.COLOR_BGR2RGB))
        output_address = "outputs/q4/p2/"+ str(j)+"_alpha_"+str(alpha)+".png"
        cv2.imwrite(output_address, np.array(out_image))
        out_image.save(p.stdin,'JPEG')

    p.stdin.close()
    p.wait()

In [20]:
img = cv2.imread("inputs/P4/kobytev_1941.png")
img2 = cv2.imread("inputs/P4/kobytev_1945.png")
images =[img, img2]
points1, points2, list_points = define_points(images)
shape = (img.shape[0],img.shape[1])
tri = create_delaunay(shape, list_points.tolist())
create_video(2, 30, img, img2, points1, points2,tri, shape, "output.mp4")
